In [1]:
import gymnasium as gym
from abc import ABC
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import defaultdict

In [2]:
def update_policy(self):
        for state in range(self.state_size):
            best_action = np.argmax(self.table[state])
            for action in range(self.action_size):
                if action == best_action:
                    self.policy[state][action] = 1 - self.epsilon + (self.epsilon / self.action_size)
                else:
                    self.policy[state][action] = self.epsilon / self.action_size

    def update_table(self, episode):
        G = 0
        visited = set()
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = self.gamma * G + reward
            
            if self.visit_type == 'first':
                if (state, action) not in visited:
                    visited.add((state, action))
                    self.returns[state][action].append(G)
                    self.table[state][action] = np.mean(self.returns[state][action])
            
            elif self.visit_type == 'every':
                self.returns[state][action].append(G)
                self.table[state][action] = np.mean(self.returns[state][action])


"def update_policy(self, episode):\n        for state in range(self.state_size):\n            best_action = np.argmax(self.table[state])\n            for action in range(self.action_size):\n                if action == best_action:\n                    self.policy[state][action] = 1 - self.epsilon + (self.epsilon / self.action_size)\n                else:\n                    self.policy[state][action] = self.epsilon / self.action_size\n        #QUESTO VIENE FATTO PER OGNI EPISODIO\n        #UPDATE RULE:\n        # Q(s_t, a_t) = Q(s_t,a_t) + v(G_t^pi - Q(s_t,a_t))\n        # con v = (W)/(C(s_t,a_t))\n        #Per ogni coppia stato-azione di un epsidio aggiorniamo G\n        # Viene calcolato al contrario partendo dalla fine andando all'inizio\n        G = 0.0\n        #IMPORTANCE SAMPLING:\n        #Andiamo a pesare la differenza tra policy esplorativa e policy target\n        W = 1.0\n        for t in reversed(range(len(episode))):\n            state, action, reward = episode[t]\n    

In [3]:
def random_policy(nA):
    return lambda observation: np.ones(nA, dtype=float) / nA

def epsilon_greedy_policy(Q, epsilon=0.1):
    def policy_fn(state):
        if isinstance(state, tuple):
            state = state[0]
        nA = len(Q[state])
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[state])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [ ]:
class tabular_agent(ABC):

    def __init__(self, env, epsilon_start, epsilon_decay, epsilon_min, episodes, gamma, visit_type):
        self.env_name = env
        self.env = gym.make(self.env_name)
        self.epsilon = epsilon_start
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.episodes = episodes
        self.gamma = gamma
        self.visit_type = visit_type
        self.action_space = self.env.action_space
        self.action_size = self.env.action_space.n
        # For FrozenLake-v1, state space is discrete, so no binning needed
        self.state_size = self.env.observation_space.n
        # Definition of the Q-table
        self.table = np.zeros([self.state_size, self.action_size])
        self.Q = defaultdict(lambda: np.zeros(env.action_space.n))
        #Contatore coppie stato-azione per mediare i ritorni incrementali
        self.C = defaultdict(lambda: np.zeros(env.action_space.n))
        self.returns = {state: {action: [] for action in range(self.action_size)} for state in range(self.state_size)}
        #Policy esplorativa
        self.behavior_policy = random_policy(env.action_space.n)
        #Policy da ottimizzare ed usare
        self.target_policy = epsilon_greedy_policy(self.Q, epsilon=self.epsilon)
        self.score = []

    def update_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def generate_episode(self):
        #3) Generazione episodio singolo
        episode = []
        state, _ = self.env.reset()
        done = False
        
        while not done:
            #action = self.select_action(state)
            action_probs = self.behavior_policy(state)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            
            next_state, reward, done, _, _ = self.env.step(action)
            episode.append((state, action, reward))
            state = next_state
        return episode
    
    def update(self, episode, episode_num):
        #QUESTO VIENE FATTO PER OGNI EPISODIO
        print(f"UPDATE EPISODE: {episode_num}\n")
        #UPDATE RULE:
        # Q(s_t, a_t) = Q(s_t,a_t) + v(G_t^pi - Q(s_t,a_t))
        # con v = (W)/(C(s_t,a_t))

        #Per ogni coppia stato-azione di un epsidio aggiorniamo G
        # Viene calcolato al contrario partendo dalla fine andando all'inizio
        G = 0.0
        visited = set()
        #IMPORTANCE SAMPLING:
        #Andiamo a pesare la differenza tra policy esplorativa e policy target
        W = 1.0
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            if isinstance(state, tuple):
                state = state[0]
            self.C[state][action] += W
            self.Q[state][action] += (W / self.C[state][action]) * (G - self.Q[state][action])
            if action != np.argmax(self.target_policy(state)):
                break
            W /= self.behavior_policy(state)[action]
    
    def update_table(self, episode):
        G = 0
        W = 1.0
        visited = set()
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = self.gamma * G + reward
            
            if self.visit_type == 'first':
                if (state, action) not in visited:
                    visited.add((state, action))
                    self.returns[state][action].append(G)
                    self.table[state][action] = np.mean(self.returns[state][action])
            
            elif self.visit_type == 'every':
                self.returns[state][action].append(G)
                self.table[state][action] = np.mean(self.returns[state][action])

    def select_action(self, state):
        return np.random.choice(np.arange(self.action_size), p=self.behavior_policy[state])

    def learn(self):
        #2) Learn, chiamato giu
        print(f"\nINIZIO LEARNING\n")
        for e in tqdm(range(self.episodes), desc="Learning"):
            episode = self.generate_episode()
            """self.update_table(episode)
            self.update_policy()"""
            self.update(episode,2)
            self.update_epsilon()
            self.score.append(sum([x[2] for x in episode]))  # Sum of rewards for this episode

    def plot_learning(self, N, title=None, filename=""):
        plt.figure()
        
        # Convert scores to numpy array
        scores = np.array(self.score)
        
        # Compute the moving average
        mean_score = np.convolve(scores, np.ones(N)/N, mode='valid')
        
        # Compute the moving variance
        variance = np.array([np.var(scores[max(0, i-N+1):i+1]) for i in range(len(scores))])
        variance = np.convolve(variance, np.ones(N)/N, mode='valid')
        
        # Adjust x values for the moving average plot
        x_values = np.arange(len(mean_score)) + (N // 2)

        # Plot the original scores
        plt.plot(scores, label='Score')
        
        # Plot the moving average
        plt.plot(x_values, mean_score, label='Moving Average', color='orange')
        
        # Plot the moving variance
        plt.plot(x_values, variance, label='Moving Variance', color='red')
        
        if title is not None:
            plt.title(title)
        plt.legend()
        plt.savefig(self.env_name + filename)
        plt.show()


    def save_model(self, path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        np.save(path, self.table)
        print(f"Q-table saved to {path}")

    def load_model(self, path):
        if os.path.exists(path):
            self.table = np.load(path)
            print(f"Q-table loaded from {path}")
        else:
            print(f"File {path} does not exist.")

    def simulate(self):
        env = gym.make(self.env_name, render_mode="human")
        self.epsilon = -1  # Set epsilon to -1 to disable exploration
        done = False
        state, _ = env.reset()
        while True:
            action = self.select_action(state)
            next_state, reward, done, _, _ = env.step(action)
            if done:
                break
            state = next_state

        env.close()


In [ ]:
if __name__ == '__main__':
    env = 'FrozenLake-v1'  # Environment set for FrozenLake
    epsilon_start = 1.0
    epsilon_decay = 0.995  # Slower decay for more episodes
    epsilon_min = 0.01
    episodes = 30000  # Adjust as needed
    gamma = 0.99
    
    # Initialize the Monte Carlo agent with either 'first' or 'every' visit type
    visit_type = 'first'  # Change to 'every' for Every-Visit Monte Carlo
    #1) Creazione agente
    agent = tabular_agent(env, epsilon_start, epsilon_decay, epsilon_min, episodes, gamma, visit_type)
    #2) Inizio Learn
    agent.learn()
    
    
    #n) Parte sui grafici
    agent.plot_learning(100, filename=f"_MonteCarlo_{visit_type}_visit")
    
    #m) Salvo modello
    agent.save_model(f"models/frozenlake_monte_carlo_{visit_type}_visit.npy")

In [ ]:
#o) Fatto modello, fai simulazione con FV
    agent.simulate()

In [ ]:
#p) Stessa cosa ma con FV
    visit_type = 'every'  # Change to 'every' for Every-Visit Monte Carlo
    agent = tabular_agent(env, epsilon_start, epsilon_decay, epsilon_min, episodes, gamma, visit_type)

    agent.learn()
    agent.plot_learning(100, filename=f"_MonteCarlo_{visit_type}_visit")

    # Save the trained model
    agent.save_model(f"models/frozenlake_monte_carlo_{visit_type}_visit.npy")

    agent.simulate()